In [1]:
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
import re
from time import sleep
import acquire_r as ar

In [2]:
url = 'https://www.pro-football-reference.com/boxscores/202011120oti.htm'
response = requests.get(url)
response

<Response [200]>

- 1 SOUP
- 2 div_grid [0]
- 3 div [0]
- 4 div_wrap [0]
- 5 soup_wrap4 [4]
- 6 soup4 : reparse soup_wrap4
- 7 wx

In [3]:
#1
soup = BeautifulSoup(response.content, 'html.parser')
#2
div_grid = soup.find_all('div', class_='content_grid')[0]
#3
div = div_grid.find_all('div')[0]
#4
div_wrap = div.find_all('div')[0]
#5
soup_wrap4 = div_wrap.contents[4]
#6
soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
#7
wx = soup4.find_all('td','center')[6]

In [4]:
wx.contents

['56 degrees, relative humidity 46%, wind 6 mph']

In [5]:
wx.contents[0]

'56 degrees, relative humidity 46%, wind 6 mph'

In [6]:
content = str(wx.contents[0])

In [ ]:
#content = str(content)

In [7]:
type(content)

str

In [8]:
content

'56 degrees, relative humidity 46%, wind 6 mph'

## So now we have options...

In [ ]:
# regex to return numerical string at start of object
temp = r'^\d{1,2}'
re.findall(temp,content)

In [ ]:
wind = r'\d{1,2}[\smph]'
# etc....

## Or, a different approach

In [9]:
wx_digits = r'\d{1,2}'
re.findall(wx_digits,content)

['56', '46', '6']

In [10]:
re.findall(wx_digits,content)[0]

'56'

In [11]:
re.findall(wx_digits,content)[0]

'56'

In [12]:
re.findall(wx_digits,content)[1]

'46'

In [13]:
re.findall(wx_digits,content)[2]

'6'

## Ok, now we increase the scale to WEEKLY

In [14]:
url = 'https://www.pro-football-reference.com/years/2020/week_10.htm'

In [15]:
response = requests.get(url)
response

<Response [200]>

In [16]:
response

<Response [200]>

In [17]:
soup = BeautifulSoup(response.content, 'html.parser')

In [18]:
html_links = soup.find_all('td', class_='right gamelink')

In [19]:
html_links[0].find('a')['href']

'/boxscores/202011120oti.htm'

## List of Games

In [20]:
[item.find('a')['href'] for item in html_links]

['/boxscores/202011120oti.htm',
 '/boxscores/202011150car.htm',
 '/boxscores/202011150cle.htm',
 '/boxscores/202011150det.htm',
 '/boxscores/202011150gnb.htm',
 '/boxscores/202011150nyg.htm',
 '/boxscores/202011150crd.htm',
 '/boxscores/202011150mia.htm',
 '/boxscores/202011150rai.htm',
 '/boxscores/202011150nor.htm',
 '/boxscores/202011150pit.htm',
 '/boxscores/202011150ram.htm',
 '/boxscores/202011150nwe.htm',
 '/boxscores/202011160chi.htm']

In [21]:
games = [item.find('a')['href'] for item in html_links]

In [22]:
games

['/boxscores/202011120oti.htm',
 '/boxscores/202011150car.htm',
 '/boxscores/202011150cle.htm',
 '/boxscores/202011150det.htm',
 '/boxscores/202011150gnb.htm',
 '/boxscores/202011150nyg.htm',
 '/boxscores/202011150crd.htm',
 '/boxscores/202011150mia.htm',
 '/boxscores/202011150rai.htm',
 '/boxscores/202011150nor.htm',
 '/boxscores/202011150pit.htm',
 '/boxscores/202011150ram.htm',
 '/boxscores/202011150nwe.htm',
 '/boxscores/202011160chi.htm']

## List of homes

In [ ]:
home_re = r'[a-z]{3}'

In [ ]:
home = re.findall(home_re,games[0])[3]

In [ ]:
type(home)

In [ ]:
home

In [ ]:
homes = [re.findall(home_re,i)[3] for i in games]

In [ ]:
# homes_2 = []
# for i in games:
#     home_re = r'[a-z]{3}'
#     home = re.findall(home_re,i)[3]
#     homes.append(home)

In [ ]:
homes

## List of seasons

In [ ]:
date_re = r'[0-9]{4}'

In [ ]:
re.findall(date_re,games[0])[0]

In [ ]:
season = re.findall(date_re,games[0])[0]

In [ ]:
season

In [ ]:
seasons = [re.findall(date_re,i)[0] for i in games]

In [ ]:
seasons

## List of weeks

In [ ]:
weeks = [1 for i in games]

In [ ]:
weeks

## List of temps

In [ ]:
temps = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    temp = re.findall(wx_digits,content)[0]
    temps.append(temp)

In [ ]:
temps

In [ ]:
df = pd.DataFrame({
    'season':seasons,
    'week':weeks,
    'home':homes,
    'temp':temps
})

In [ ]:
df

## List of winds

In [25]:
response

<Response [200]>

In [24]:
winds = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    wind = re.findall(wx_digits,content)[2]
    winds.append(wind)

IndexError: list index out of range

In [ ]:
winds

## List of hums

In [ ]:
response

In [ ]:
hums = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    hum = re.findall(wx_digits,content)[1]
    hums.append(hum)

In [ ]:
hums

In [ ]:
df = pd.DataFrame({
    'schedule_season':seasons,
    'schedule_week':weeks,
    'team_home':homes,
    'weather_temperature':temps,
    'weather_wind_mph':winds,
    "weather_humidity":hums
})

In [ ]:
df

## Write to .csv

In [ ]:
df.to_csv('season_2022_w1.csv', index=False)

---
---
## NEXT WEEK

In [ ]:
url = 'https://www.pro-football-reference.com/years/2022/week_2.htm'

In [ ]:
response = requests.get(url)

In [ ]:
response

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')
html_links = soup.find_all('td', class_='right gamelink')
games = [item.find('a')['href'] for item in html_links]

In [ ]:
games

## List of homes

In [ ]:
home_re = r'[a-z]{3}'
homes = [re.findall(home_re,i)[3] for i in games]

In [ ]:
homes

## List of seasons

In [ ]:
date_re = r'[0-9]{4}'
seasons = [re.findall(date_re,i)[0] for i in games]

In [ ]:
seasons

## List of weeks

In [ ]:
weeks = [2 for i in games]

In [ ]:
weeks

## List of temps

In [ ]:
temps = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    temp = re.findall(wx_digits,content)[0]
    temps.append(temp)

In [ ]:
temps

## List of winds

In [ ]:
response

In [ ]:
winds = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    wind = re.findall(wx_digits,content)[2]
    winds.append(wind)

In [ ]:
winds

## List of hums

In [ ]:
response

In [ ]:
hums = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    hum = re.findall(wx_digits,content)[1]
    hums.append(hum)

In [ ]:
hums

In [ ]:
df = pd.DataFrame({
    'schedule_season':seasons,
    'schedule_week':weeks,
    'team_home':homes,
    'weather_temperature':temps,
    'weather_wind_mph':winds,
    "weather_humidity":hums
})

In [ ]:
df

## Write to .csv

In [ ]:
df.to_csv('season_2022_w2.csv', index=False)

---
---
## NEXT WEEK with SLEEEEEEEEEEEP......

In [ ]:
url = 'https://www.pro-football-reference.com/years/2022/week_3.htm'
response = requests.get(url)

In [ ]:
response

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')
html_links = soup.find_all('td', class_='right gamelink')
games = [item.find('a')['href'] for item in html_links]
games

## List of homes

In [ ]:
home_re = r'[a-z]{3}'
homes = [re.findall(home_re,i)[3] for i in games]
homes

## List of seasons

In [ ]:
date_re = r'[0-9]{4}'
seasons = [re.findall(date_re,i)[0] for i in games]
seasons

## List of weeks

In [ ]:
weeks = [3 for i in games]
weeks

## List of temps

In [ ]:
temps = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    temp = re.findall(wx_digits,content)[0]
    temps.append(temp)

In [ ]:
temps

## List of winds

In [ ]:
response

In [ ]:
winds = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    wind = re.findall(wx_digits,content)[2]
    winds.append(wind)

In [ ]:
winds

## List of hums

In [ ]:
response

In [ ]:
hums = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    hum = re.findall(wx_digits,content)[1]
    hums.append(hum)

In [ ]:
hums

In [ ]:
df = pd.DataFrame({
    'schedule_season':seasons,
    'schedule_week':weeks,
    'team_home':homes,
    'weather_temperature':temps,
    'weather_wind_mph':winds,
    "weather_humidity":hums
})

In [ ]:
df

## Write to .csv

In [ ]:
df.to_csv('season_2022_w3.csv', index=False)

## List of home_teams

In [ ]:
url = 'https://www.pro-football-reference.com/boxscores/202210020nor.htm'
response = requests.get(url)

In [ ]:
response

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
href = soup.find_all('div', class_='scorebox')[0].find_all('a')[8]

In [ ]:
href

In [ ]:
href.text

In [ ]:
team = href.text

In [ ]:
response

In [ ]:
wk = [str(i) for i in range(1,23)]

In [ ]:
wk

## List of weeks

In [ ]:
url = 'https://www.pro-football-reference.com/boxscores/202210090gnb.htm'
response = requests.get(url)

In [ ]:
response

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
span = soup.find_all('div', class_='section_wrapper')[0].find_all('span')[0]

In [ ]:
span

In [ ]:
span = str(span)

In [ ]:
span

In [ ]:
week_re = r'[0-9]{1,2}'
week = re.findall(week_re,span)

In [ ]:
week[0]

In [ ]:
week_list = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    span = soup.find_all('div', class_='section_wrapper')[0].find_all('span')[0]
    span = str(span)
    week_re = r'[0-9]{1,2}'
    week = re.findall(week_re,span)
    week = week[0]
    week_list.append(week)

In [ ]:
week_list